In [1]:
import skimage.io as io
import skimage as sk
from skimage.color import rgb2gray
from skimage.transform import resize
import time
import numpy as np
import os


from unitTesting import UnitTest
import adaboost.adaboost as ab

# Testing

In [2]:
img = io.imread('./test.jpg')
img = rgb2gray(img)
ut = UnitTest(img)

#load positive dataset
try:
    print("Loading positive dataset...")
    positiveDataset = np.load("./Dataset/positiveDataset.npy")
except:
    print("No dataset found, creating new dataset...")
    positiveDataset = np.load("./Dataset/olivetti_faces.npy")
    personID = np.load("./Dataset/olivetti_faces_target.npy")


    #resize images to 24x24
    newPositiveDataset = np.empty(shape=(positiveDataset.shape[0], 24, 24))
    for i in range(len(positiveDataset)):
        newImg = np.array(positiveDataset[i])
        
        finalImg = resize(newImg, (24, 24))
        newPositiveDataset[i] = finalImg

    positiveDataset = newPositiveDataset

    # for i in range(10):
    #     io.imshow(positiveDataset[i])
    #     io.show()

    # print(positiveDataset.shape)

    

    np.save("./Dataset/positiveDataset.npy", positiveDataset)

#loading negative dataset
try:
    print("Loading negative dataset...")
    negativeDataset = np.load("./Dataset/negativeDataset.npy")
except:
    print("No dataset found, creating new dataset...")
    directory = "./Dataset/NegativeSet"
    negativeDataset = np.empty(shape=(len(os.listdir(directory)), 24, 24))
    i = 0
    for filename in os.listdir(directory):
        if filename.endswith(".jpg"):
            curImg = io.imread(os.path.join(directory, filename), as_gray=True)
            newImg = resize(curImg, (24, 24))
            negativeDataset[i] = newImg
            i += 1
    # negativeDataset = negativeDataset.reshape(len(os.listdir(directory)), 64, 64)
    # print(negativeDataset.shape)
    np.save("./Dataset/negativeDataset.npy", negativeDataset)



posDataset, negDataset = ab.preprocessImages(positiveDataset, negativeDataset)

print(f"Positive Dataset Shape: {posDataset.shape}")
print(f"Negative Dataset Shape: {negDataset.shape}")



Integral image test PASSED
Determine features test PASSED
Loading positive dataset...
Loading negative dataset...
Positive Dataset Shape: (118, 24, 24)
Negative Dataset Shape: (11, 24, 24)


In [3]:
minFeatureWidth = 4
maxFeatureWidth = 20
minFeatureHeight = 4
maxFeatureHeight = 20

classifiersToBeUsed = []
try :
    print("Loading classifiers...")
    classifiersToBeUsed = np.load("./classifiers.npy", allow_pickle=True)
except Exception as e:
    # print(e)
    print("No classifiers found, creating new classifiers...")
    classifier = ab.AdaBoost()
    nClassifiers = 50
    start = time.time()
    classifiersToBeUsed = classifier.learn(posDataset, negDataset,0.3,minFeatureWidth,minFeatureHeight,maxFeatureWidth,maxFeatureHeight, nClassifiers)
    end = time.time()
    classifiersToBeUsed = np.array(classifiersToBeUsed)
    np.save("./classifiers.npy", classifiersToBeUsed)
    print("Time taken to train the classifier: ", end-start)

print("Classifiers loaded successfully")

testDataSet = np.load("./Dataset/testFaces.npy")
testDataSetTarget = np.load("./Dataset/testTargets.npy")
#print the first 10 images in the test dataset
# for i in range(10):
#     io.imshow(testDataSet[i])
#     io.show()


i = 0
correctCount = 0
for test in testDataSet:
    predicted = ab.getVotes(np.array(classifiersToBeUsed), test)
    if predicted == testDataSetTarget[i]:
        correctCount += 1
    i += 1

print("Accuracy: ", correctCount/len(testDataSet) * 100, "%")








Loading classifiers...
No classifiers found, creating new classifiers...
Calculating integral images...
Done!

Creating initial weights and labels...
Done!

Calculating all possible haar features...
Done!

Calculating votes of all haar features for all samples...
Done!

Training 218702 classifiers...
 Training classifier 1 of 50...

d:\University\Senior 2\Fall 2022\Image Processing\Project\FaceDetection-Recognition\Detection\adaboost\adaboost.py:245: RuntimeWarning: divide by zero encountered in log
  bestFeature.weight = 0.5 * np.log((1-bestError)/bestError)
d:\University\Senior 2\Fall 2022\Image Processing\Project\FaceDetection-Recognition\Detection\adaboost\adaboost.py:253: RuntimeWarning: invalid value encountered in divide
  weights = weights/np.sum(weights)


 Training classifier 50 of 50...Time taken to train the classifier:  290.2776527404785
Classifiers loaded successfully
Error1: Feature out of bounds
Error2: Feature out of bounds
x: 0, y: 22, width: 4, height: 2, intImg.shape: (24, 24)


IndexError: index 24 is out of bounds for axis 0 with size 24